# Сливание текстов для TM

Что решили

1. Берем только тексты после 20 октября 2010
2. Выкидываем авторов, у которых за этот период менее 50 слов
3. Авторов, у которых менее 300 слов, сливаем целиком за весь период
4. Авторов, у которых более 300 слов:
   1. Режем на куски примерно по 300 слов, с учетом границ текстов;
   2. Если при такой нарезке последний кусок оказывается менее 50 слов, добавляем к нему предпоследний кусок и делим примерно пополам, с учетом границ текстов.

посмотрим, что получится
следующий вариант: сливаем каждого автора целиком. смотрим, не получается ли размазка текстов по темам хуже, чем в первом варианте. если нет, второй вариант более легко интерпретируем.

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle
import shelve
import klepto
from gensim import corpora, models
import feather
from collections import defaultdict
import itertools
import csv

In [2]:
def print_full(x):
    pd.set_option("display.max_rows", len(x))
    print(x)
    pd.reset_option("display.max_rows")

In [10]:
data = pd.read_pickle("/Users/Hun/hse_thesis/hse_thesis1.pickle")

In [4]:
data.head()

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
0,general lee,49154141,49154141,2012-02-24 11:54:20,Дмитрий,Филиппов,Владимир,Владимирская область,2,2404,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
1,shelby,49154141,49154141,2012-02-24 11:54:33,Дмитрий,Филиппов,Владимир,Владимирская область,1,2405,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
2,ford mustang,49154141,49154141,2012-02-25 22:27:35,Дмитрий,Филиппов,Владимир,Владимирская область,2,2411,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
3,murcielago,49154141,49154141,2012-02-24 11:53:51,Дмитрий,Филиппов,Владимир,Владимирская область,1,2400,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
4,суровый водитель,49154141,49154141,2012-02-23 16:22:56,Дмитрий,Филиппов,Владимир,Владимирская область,2,2392,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0


Размерность данных:

In [5]:
data.shape

(5392586, 22)

Смотрим количество пропущенных данных:

In [6]:
data.isnull().sum()

text                        0
author_id                   0
wall_id                     0
date                        0
name                    48759
surname                 49346
city                   411426
region                 421852
text_length                 0
post_comment_id_int         0
is_post                     0
sex                    184556
posts                  184556
comments               184556
likes                  184556
reposts                184556
res_postlikes          184556
res_comments           184556
university             184556
freinds                263451
groups                 263451
followers              263451
dtype: int64

Ограничиваем данные постами, написанными позже 20 октября 2010 года и считаем их количество:

In [7]:
data_good_date = data[data.date >= datetime.date(year=2010,month=10,day=21)]
data_good_date.shape

(5121979, 22)

ВНИМАНИЕ! Есть проблема: существует пользователь с id 0, на которого приходится 200477 постов. В него слились все кривозакачанные пользователи. Надо будет его исключать. Пока ничего с ним не делаю.

ИЛИ! Если считать посты не по пользоватлеям, а по стенам, то ничего страшного, потому что у нулевых пользователей есть нормальная метка стены. Но на стене встречаются посты пользователей из разных регионов, поэтому не ясно, к какому региону относить слитые тексты.

In [8]:
zero_users = data[data["author_id"] == 0]
zero_users.shape

(232859, 22)

In [9]:
zero_users

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
56,пап понравиться мама понравиться светлана серега,0,44588833,2013-11-25 05:53:48,Sergey,Tsygankov,Барнаул,Курганская область,6,1359,...,107.0,28.0,36.0,71.0,53.0,1.0,,115.0,35.0,13.0
258,проверять,0,44588833,2013-11-14 03:49:11,Sergey,Tsygankov,Барнаул,Курганская область,1,1330,...,107.0,28.0,36.0,71.0,53.0,1.0,,115.0,35.0,13.0
441,мда,0,44588833,2013-11-15 09:53:47,Svetlana,Svetlanova,NaN,NaN,1,1307,...,1682.0,63.0,506.0,14.0,725.0,0.0,,65.0,286.0,24.0
626,план понедельник пить вечер,0,44588833,2013-10-21 03:15:14,Sergey,Tsygankov,Барнаул,Курганская область,4,1270,...,107.0,28.0,36.0,71.0,53.0,1.0,,115.0,35.0,13.0
645,ничё прорываться,0,44588833,2013-10-21 03:14:49,Sergey,Tsygankov,Барнаул,Курганская область,2,1269,...,107.0,28.0,36.0,71.0,53.0,1.0,,115.0,35.0,13.0
714,прикол коврик сшивать подарить светлана замета...,0,44588833,2013-10-21 03:14:12,Sergey,Tsygankov,Барнаул,Курганская область,8,1267,...,107.0,28.0,36.0,71.0,53.0,1.0,,115.0,35.0,13.0
805,хаааа,0,38899395,2012-01-23 20:33:04,Кирилл,Тулкин,Александров,Владимирская область,1,256,...,1.0,0.0,5.0,1.0,4.0,0.0,,148.0,186.0,54.0
808,vk дашенька аахахахахах,0,38899395,2015-01-02 12:21:20,Дмитрий,Абатнин,Карабаново,Белгородская область,3,1484,...,1.0,1.0,3.0,2.0,27.0,0.0,РосНОУ (АФ),108.0,119.0,22.0
809,vk анатолий,0,38899395,2015-07-01 22:51:51,Вадим,Романов,Александров,Владимирская область,2,1471,...,5.0,4.0,6.0,9.0,53.0,0.0,,204.0,80.0,21.0
815,жесть билет бесплатный ваня бесплатный,0,38899395,2014-12-16 14:08:46,Вадим,Романов,Александров,Владимирская область,5,1453,...,5.0,4.0,6.0,9.0,53.0,0.0,,204.0,80.0,21.0


Определяем пользователей, у которых 50 и более слов за рассчётный период. Их 42631.

In [10]:
groupedby_author_sum = data_good_date.groupby("author_id").agg({"text_length": "sum"})
groupedby_author_sum_50 = groupedby_author_sum[groupedby_author_sum["text_length"] >= 50]
authors_50 = groupedby_author_sum_50.index.values
len(authors_50)

42631

Создадим данные только с этими пользователями. Осталось 4414265 документов.

In [11]:
data_authors_50 = data_good_date[data_good_date["author_id"].isin(authors_50)]
len(data_authors_50)

4414265

Посмотрим уникальные регионы

In [12]:
data_authors_50["region"].value_counts()

Тюменская область                  202486
Пензенская область                 139186
Севастопольский городской совет    127923
Крым область                       100095
Дагестан                            98537
Орловская область                   91166
Архангельская область               90603
Коми                                90374
Ленинградская область               89177
Московская область                  87213
Краснодарский край                  83567
Мордовия                            78327
Санкт-Петербург                     75861
Москва                              74280
Ставропольский край                 72514
Вологодская область                 72444
Карелия                             72401
Псковская область                   71511
Нижегородская область               71079
Новосибирская область               70385
Северная Осетия - Алания            67505
Воронежская область                 67217
Татарстан                           64926
Хакасия                           

И возьмём из них те, из которых у нас есть более 10000 записей. Таким образом, осталось 4163945 записей.

In [13]:
valid_regions = data_authors_50["region"].value_counts()[data_authors_50["region"].value_counts() > 10000].axes[0].tolist()

In [14]:
data_authors_50 = data_authors_50[data_authors_50["region"].isin(valid_regions)]

In [15]:
data_authors_50.isnull().sum()

text                        0
author_id                   0
wall_id                     0
date                        0
name                        0
surname                     0
city                        0
region                      0
text_length                 0
post_comment_id_int         0
is_post                     0
sex                     84463
posts                   84463
comments                84463
likes                   84463
reposts                 84463
res_postlikes           84463
res_comments            84463
university              84463
freinds                150747
groups                 150747
followers              150747
dtype: int64

In [16]:
data_authors_50.shape

(4163945, 22)

In [17]:
data_authors_50["city"].value_counts()

Тюмень                164723
Севастополь           131802
Пенза                 107088
Санкт-Петербург        75861
Москва                 74302
Махачкала              72941
Орёл                   70289
Новосибирск            63741
Чита                   52206
Воронеж                50176
Саранск                49813
Омск                   48206
Липецк                 46908
Симферополь            45302
Смоленск               43842
Ульяновск              42833
Кострома               42562
Владикавказ            41596
Нижний Новгород        41213
Томск                  40619
Ростов-на-Дону         39491
Псков                  38057
Калуга                 37639
Ярославль              37184
Самара                 36805
Казань                 36726
Киров                  36075
Петрозаводск           35475
Краснодар              35430
Архангельск            34753
                       ...  
Емуртла                    1
Старобачаты                1
Усть-Омчуг                 1
Светлый       

Авторов, у которых менее 300 слов, сливаем целиком за весь период.

Определяем таких авторов.

In [18]:
groupedby_author_sum = data_authors_50.groupby("author_id").agg({"text_length": "sum"})
authors_300 = groupedby_author_sum[groupedby_author_sum["text_length"] <= 300].index.values

In [19]:
data_authors_300 = data_authors_50[data_authors_50["author_id"].isin(authors_300)]
data_authors_300.shape

(346635, 22)

In [20]:
merged_300 = data_authors_300.groupby("author_id").agg({"text": "sum", "region": "first", "city": "first",
                                                        "sex": "first", "freinds": "first", "followers": "first",
                                                       "res_postlikes": "first", "likes": "first", "groups": "first",
                                                        })
merged_300["text"] = merged_300["text"].apply(lambda x: [x])

merged_300 — слитые по каждому автору посты авторов, написавших от 50 до 300 слов включительно за рассчётный период. Таких авторов 23877.

In [21]:
merged_300.shape

(23877, 9)

In [22]:
merged_300.head()

,likes,city,sex,freinds,res_postlikes,text,followers,groups,region
author_id,,,,,,,,,
23835,3.0,Санкт-Петербург,1,568.0,0.0,[придти рыбка vk придти листик vk придти листи...,1341.0,59.0,Санкт-Петербург
47882,2.0,Санкт-Петербург,2,120.0,2.0,[машин ваз довольный почесть устраивать устраи...,228.0,38.0,Санкт-Петербург
78076,30.0,Вологда,1,372.0,13.0,[прибывать удача экзамен юный кинуть подношени...,3065.0,49.0,Вологодская область
106198,0.0,Санкт-Петербург,2,626.0,0.0,[викторий отправлять подарок энергия игра тайн...,1207.0,65.0,Санкт-Петербург
107733,16.0,Санкт-Петербург,2,160.0,26.0,[помнить жизнь повторять огнь день победа vkon...,8.0,24.0,Санкт-Петербург


Для авторов, у которых более 300 слов:

Берём каждого пользователя. Последовательно сливаем его тексты и проверям количество слов. Если в какой-то момент это количество достигло или превысило 300, формируем документ. Затем формируем следующий документ.

Если при такой нарезке последний кусок оказыается менее 50 слов, его к предыдущему.

In [23]:
groupedby_author_sum = data_authors_50.groupby("author_id").agg({"text_length": "sum"})
authors_more_300 = groupedby_author_sum[groupedby_author_sum["text_length"] > 300].index.values

In [24]:
data_authors_more_300 = data_authors_50[data_authors_50["author_id"].isin(authors_more_300)]
data_authors_more_300.shape

(3817310, 22)

Удаляем пользователей с id = 0.

In [25]:
data_authors_more_300 = data_authors_more_300[data_authors_more_300["author_id"] != 0]
data_authors_more_300.shape

(3616833, 22)

In [26]:
data_authors_more_300.head()

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
0,general lee,49154141,49154141,2012-02-24 11:54:20,Дмитрий,Филиппов,Владимир,Владимирская область,2,2404,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
1,shelby,49154141,49154141,2012-02-24 11:54:33,Дмитрий,Филиппов,Владимир,Владимирская область,1,2405,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
2,ford mustang,49154141,49154141,2012-02-25 22:27:35,Дмитрий,Филиппов,Владимир,Владимирская область,2,2411,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
3,murcielago,49154141,49154141,2012-02-24 11:53:51,Дмитрий,Филиппов,Владимир,Владимирская область,1,2400,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
4,суровый водитель,49154141,49154141,2012-02-23 16:22:56,Дмитрий,Филиппов,Владимир,Владимирская область,2,2392,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0


In [27]:
from collections import defaultdict

# result = {author_id: {docs: [("text", likes), ...], probs: [(t_num, prob)...], region: "str", age: "str", sex: int}, ...}

merged_more_300 = {}
for author_id, author_data in data_authors_more_300.groupby("author_id"):
    texts_storage = ""
    texts_number = len(author_data["text"])
    
    auth_dict = {}
    auth_dict["docs"] = []
    auth_dict["region"] = author_data["region"].iloc[0]
    auth_dict["city"] = author_data["city"].iloc[0]
    auth_dict["sex"] = author_data["sex"].iloc[0]
    auth_dict["freinds"] = author_data["freinds"].iloc[0]
    auth_dict["followers"] = author_data["followers"].iloc[0]
    auth_dict["likes"] = author_data["likes"].iloc[0]
    auth_dict["res_postlikes"] = author_data["res_postlikes"].iloc[0]
    auth_dict["groups"] = author_data["groups"].iloc[0]

    
    for text_num, text in enumerate(author_data["text"]):
        last = (text_num + 1) == texts_number
        texts_storage += text
        
        if last:
            if len(texts_storage.split()) >= 50:
                # если это последний документ и количество слов в нём превышает 50 — формируем документ и формируем автора
                auth_dict["docs"].append(texts_storage)
                merged_more_300[author_id] = auth_dict
                texts_storage = "" 
            else:
                # а если не превышает — добавляем к предыдущему и формируем автора
                auth_dict["docs"][-1] += texts_storage
                merged_more_300[author_id] = auth_dict
                texts_storage = "" 
        else:
            if len(texts_storage.split()) >= 300:
                # если количество слов в документе достигло 300 — формируем документ
                auth_dict["docs"].append(texts_storage)
                texts_storage = ""
            else:
                # если ни одно условие не выполнено, продолжаем сливать
                continue

In [28]:
merged_300.rename(columns={"text": "docs"}, inplace=True)
merged_300_dict = merged_300.to_dict(orient="index")

In [29]:
len(merged_300_dict)

23877

In [30]:
#texts = [item for sublist in merged_texts.values() for item in sublist]
#texts += merged_300["text"].tolist()
general_dict = {**merged_more_300, **merged_300_dict}
len(general_dict)

36396

In [31]:
list(general_dict.items())[1]

(112984077,
 {'city': 'Визинга',
  'docs': ['март красивый картинка открытка vk fotomimi наступать март красивый картинка открытка vk fotomimi отправлять vk top cards отправлять vk отправлять vk отправлять огромныеоткрытки vk request key отправлять огромный открытка рфhttps vk top cards mid loc отправлять огромный открытка рфhttps vk top cards mid loc vkontakte magic поздравлять небольшой сюрприз vkontakte postcard post отправлять огромный открытка рфhttps vk top cards mid loc отправлять огромный открытка рфhttps vk top cards mid loc подарить уникальный подарок vk mid loc ольга подбирать город котор комфортно жить vkontakte sovet youhttps vk ольга фото понравиться какое vkontakte foto likehttps vk foto mid ольга фото понравиться какое vkontakte foto likehttps vk foto mid '],
  'followers': 55.0,
  'freinds': 120.0,
  'groups': 8.0,
  'likes': 0.0,
  'region': 'Коми',
  'res_postlikes': 12.0,
  'sex': 1})

In [32]:
with open("/Users/Hun/hse_thesis/merged_texts_with_meta.pickle", "wb") as f:
    pickle.dump(general_dict, f)

In [3]:
general_dict = pickle.load(open("/Users/Hun/hse_thesis/merged_texts_with_meta.pickle", "rb"))

Итоговое количество текстов: 239732, было 254432

In [33]:
texts = []
for auth_id, val in general_dict.items():
    for text in val["docs"]:
        texts.append(text)
len(texts)

239732

Количество уникальных слов в корпусе

In [34]:
uniq_words = set()
for text in texts:
    words_in_text = set(text.split())
    uniq_words.update(words_in_text)
print(len(uniq_words))

225450


Конвертация текстов для Topic Miner

In [8]:
for num, text in enumerate(texts):
    with open("/Users/Hun/hse_thesis/texts_for_tm_2/{}.txt".format(str(num + 1)), "wt") as f:
        f.write(text)

In [35]:
dictionary = corpora.Dictionary([text.split() for text in texts])

In [36]:
dictionary.save("/Users/Hun/hse_thesis/hse_thesis.dict_valid_regions")

In [5]:
dictionary = corpora.Dictionary.load("/Users/Hun/hse_thesis/hse_thesis.dict_valid_regions")

In [37]:
corpus = [dictionary.doc2bow(text.split()) for text in texts]
corpora.MmCorpus.serialize("/Users/Hun/hse_thesis/hse_thesis.corpus_valid_regions", corpus, metadata=True)

In [37]:
corpus = corpora.MmCorpus("/Users/Hun/hse_thesis/hse_thesis.corpus_valid_regions")

In [38]:
lda150 = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=150)

In [39]:
lda150.save("/Users/Hun/hse_thesis/lda150/hse_thesis.model150")

In [3]:
lda150 = models.LdaModel.load("/Users/Hun/hse_thesis/lda150/hse_thesis.model150")

In [40]:
lda150.print_topics(num_topics=-1, num_words=10)

[(0,
  '0.017*рассада + 0.012*арбуз + 0.009*абонемент + 0.009*втирать + 0.009*полезно + 0.009*нога + 0.009*рука + 0.008*чеснок + 0.008*грунт + 0.007*кустик'),
 (1,
  '0.024*война + 0.023*pikabu + 0.022*story + 0.015*бой + 0.015*солдат + 0.012*армия + 0.011*победа + 0.009*великий + 0.008*военный + 0.008*оружие'),
 (2,
  '0.008*хороший + 0.007*мужик + 0.007*спать + 0.006*пойти + 0.005*ночь + 0.005*сидеть + 0.005*пот + 0.005*любить + 0.005*пить + 0.005*видеть'),
 (3,
  '0.040*вод + 0.012*вода + 0.011*использовать + 0.010*раствор + 0.007*добавлять + 0.007*хороший + 0.006*пятно + 0.005*способ + 0.005*уксус + 0.005*мыло'),
 (4,
  '0.027*совершать + 0.024*небеса + 0.021*заклинание + 0.020*медаль + 0.017*коллекционер + 0.014*диана + 0.014*ранг + 0.010*анна + 0.010*ра + 0.010*лорд'),
 (5,
  '0.056*волос + 0.043*кожа + 0.036*маска + 0.034*масло + 0.022*лицо + 0.016*наносить + 0.016*ложка + 0.012*смывать + 0.011*голов + 0.011*минута'),
 (6,
  '0.024*кирилл + 0.008*чан + 0.008*шпион + 0.007*ракетн

In [41]:
lda150.minimum_probability = 0

In [42]:
for user_id, val in general_dict.items():
    general_dict[user_id]["probs"] = []
    for doc in val["docs"]:
        probs = lda150[dictionary.doc2bow(doc.split())]
        general_dict[user_id]["probs"].append([(t_num, prob) for t_num, prob in probs])

In [43]:
general_dict[112984077]

{'city': 'Визинга',
 'docs': ['март красивый картинка открытка vk fotomimi наступать март красивый картинка открытка vk fotomimi отправлять vk top cards отправлять vk отправлять vk отправлять огромныеоткрытки vk request key отправлять огромный открытка рфhttps vk top cards mid loc отправлять огромный открытка рфhttps vk top cards mid loc vkontakte magic поздравлять небольшой сюрприз vkontakte postcard post отправлять огромный открытка рфhttps vk top cards mid loc отправлять огромный открытка рфhttps vk top cards mid loc подарить уникальный подарок vk mid loc ольга подбирать город котор комфортно жить vkontakte sovet youhttps vk ольга фото понравиться какое vkontakte foto likehttps vk foto mid ольга фото понравиться какое vkontakte foto likehttps vk foto mid '],
 'followers': 55.0,
 'freinds': 120.0,
 'groups': 8.0,
 'likes': 0.0,
 'probs': [[(0, 6.2305295950155882e-05),
   (1, 6.2305295950155882e-05),
   (2, 6.2305295950155882e-05),
   (3, 6.2305295950155882e-05),
   (4, 6.230529595015

Строки: тексты
Столбцы: id автора, пол автора, регион ав, лайки, [значения разных характеристик регионов], [вероятность темы 1 — вероятность темы 150] 

In [44]:
with open("/Users/Hun/hse_thesis/region_topics.csv", "wt", newline="") as f:
    fieldnames = ["author_id", "region", "city", "groups", "likes", "sex", "res_postlikes",
                  "freinds", "followers"] + ["topic_" + str(num) for num in range(150)]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for auth_id, val in general_dict.items():
        for doc in val["probs"]:
            row_dict = {
                "author_id": auth_id,
                "region": val["region"],
                "city": val["city"],
                "groups": val["groups"],
                "likes": val["likes"],
                "sex": val["sex"],
                "res_postlikes": val["res_postlikes"],
                "freinds": val["freinds"],
                "followers": val["followers"],
            }
            t_dict = {"topic_" + str(t_num): prob for t_num, prob in doc}
            row_dict.update(t_dict)
            writer.writerow(row_dict)

In [3]:
topics_data = pd.read_csv("/Users/Hun/hse_thesis/region_topics.csv")

In [46]:
topics_data.head()

,author_id,region,city,groups,likes,sex,res_postlikes,freinds,followers,topic_0,...,topic_140,topic_141,topic_142,topic_143,topic_144,topic_145,topic_146,topic_147,topic_148,topic_149
0,3276812,Тюменская область,Тюмень,225.0,0.0,1.0,0.0,183.0,13.0,0.000103,...,0.000103,0.000103,0.000103,0.000103,0.000103,0.000103,0.000103,0.000103,0.000103,0.000103
1,112984077,Коми,Визинга,8.0,0.0,1.0,12.0,120.0,55.0,0.000062,...,0.000062,0.025407,0.000062,0.000062,0.000062,0.110751,0.000062,0.000062,0.000062,0.000062
2,143392783,Кировская область,Киров,262.0,178.0,1.0,1465.0,148.0,164.0,0.000020,...,0.000020,0.011057,0.005023,0.000020,0.017265,0.000020,0.000020,0.000020,0.000020,0.000020
3,143392783,Кировская область,Киров,262.0,178.0,1.0,1465.0,148.0,164.0,0.000022,...,0.000022,0.000022,0.000022,0.000022,0.000022,0.006104,0.000022,0.000022,0.000022,0.000022
4,143392783,Кировская область,Киров,262.0,178.0,1.0,1465.0,148.0,164.0,0.000019,...,0.000019,0.000019,0.000019,0.000019,0.000019,0.000019,0.000019,0.000019,0.000019,0.000019


Выраженность темы

In [47]:
mean_probs = defaultdict(int)

for num, topic in [(i, "topic_{}".format(i)) for i in range(150)]:
    mean = topics_data[topic].mean()
    mean_probs[num] = mean

In [48]:
joint_probs = defaultdict(int)

topics_names = [(i, "topic_{}".format(i)) for i in range(150)]
pairs = itertools.combinations(topics_names, 2)
for pair in pairs:
    joint_prob = topics_data[pair[0][1]] * topics_data[pair[1][1]]
    joint_probs[(pair[0][0], pair[1][0])] = joint_prob.mean()

In [49]:
joint_probs_norm = defaultdict(int)

topics_names = [(i, "topic_{}".format(i)) for i in range(150)]
pairs = itertools.combinations(topics_names, 2)
for pair in pairs:
    joint_prob = topics_data[pair[0][1]] * topics_data[pair[1][1]]/(mean_probs[pair[0][0]] * mean_probs[pair[1][0]])
    joint_probs_norm[(pair[0][0], pair[1][0])] = joint_prob.mean()

In [50]:
joint_probs_norm

defaultdict(int,
            {(39, 70): 0.7980148895196828,
             (80, 125): 0.62310430998113786,
             (13, 108): 0.87984620121585988,
             (100, 112): 0.84594216764435803,
             (63, 76): 0.99220607376281822,
             (29, 44): 0.2921405953451876,
             (67, 137): 0.91040662207893053,
             (33, 41): 0.16212286261375211,
             (104, 130): 1.045849664803195,
             (52, 98): 1.3460880877739949,
             (15, 30): 0.78401924802887801,
             (56, 101): 1.4439937708665613,
             (19, 91): 1.1863386199867831,
             (38, 149): 0.82290062627595995,
             (76, 88): 1.0085574590829967,
             (42, 88): 2.4278248867301429,
             (5, 84): 1.0812780411163081,
             (86, 129): 0.66895387528851558,
             (15, 143): 1.0870442611983542,
             (8, 87): 2.6922466994430545,
             (85, 87): 0.97979365846208655,
             (32, 77): 1.2226406969744765,
             (14, 7

In [51]:
pickle.dump(joint_probs_norm, open("/Users/Hun/hse_thesis/lda150/joint_probs_norm_150.pickle", "wb"))

In [52]:
pickle.dump(mean_probs, open("/Users/Hun/hse_thesis/lda150/mean_probs_150.pickle", "wb"))
pickle.dump(joint_probs, open("/Users/Hun/hse_thesis/lda150/joint_probs_150.pickle", "wb"))

In [53]:
joint_probs

defaultdict(int,
            {(39, 70): 2.4693469441255136e-06,
             (80, 125): 4.5757643188759115e-06,
             (13, 108): 4.8155061086555362e-06,
             (100, 112): 4.7859444320579748e-06,
             (63, 76): 1.8776276491771689e-05,
             (29, 44): 7.6260291351418253e-06,
             (67, 137): 4.8587408768122311e-06,
             (33, 41): 4.6548288091930204e-05,
             (104, 130): 4.3766615174087857e-06,
             (52, 98): 2.2882823038075771e-05,
             (15, 30): 1.7743874516712392e-05,
             (56, 101): 6.7043692950387651e-06,
             (19, 91): 2.7754681299950747e-06,
             (38, 149): 2.2221274109930822e-06,
             (76, 88): 1.8993666554970147e-05,
             (42, 88): 7.8106502720622591e-06,
             (5, 84): 7.9244685675229488e-05,
             (86, 129): 1.2220447091292406e-05,
             (15, 143): 3.8968831181750824e-05,
             (8, 87): 2.8804860200033746e-05,
             (85, 87): 2.305938306

In [5]:
topics_data["sex"].value_counts()

1.0    162495
2.0     71333
0.0         1
Name: sex, dtype: int64

In [12]:
data[data["sex"] == 1]

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
294,камера зашибаться луна клевый анон,233626208,233626208,2015-09-27 11:33:10,Светлана,Абрамова,Александров,Владимирская область,5,250,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
300,мама заменять заменять смочь,233626208,233626208,2015-09-22 22:56:41,Светлана,Абрамова,Александров,Владимирская область,4,247,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
301,успокоительный капли таблетка злить,233626208,233626208,2015-09-22 22:50:42,Светлана,Абрамова,Александров,Владимирская область,4,243,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
304,шрам полностью покрывать видеть заглядывать душ,233626208,233626208,2015-09-22 22:56:09,Светлана,Абрамова,Александров,Владимирская область,6,246,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
305,смеяться громкий хотеться плакать,233626208,233626208,2015-09-22 22:53:04,Светлана,Абрамова,Александров,Владимирская область,4,245,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
311,dj анн зажигать клуб гравити фолз трек blue sk...,292018487,233626208,2015-09-25 16:48:13,Аня,Колмакова,Александров,Владимирская область,16,249,...,1.0,0.0,0.0,0.0,0.0,0.0,,76.0,10.0,9.0
315,легкий обижать мать обида замечать простужатьс...,233626208,233626208,2015-09-22 22:52:45,Светлана,Абрамова,Александров,Владимирская область,16,244,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
317,домашний пятновыводитель чудесный средство рец...,233626208,233626208,2015-06-09 00:33:53,Светлана,Абрамова,Александров,Владимирская область,29,231,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
318,мужчина испытывать душевный подъедать прилив с...,233626208,233626208,2015-08-31 21:51:27,Светлана,Абрамова,Александров,Владимирская область,15,223,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
320,маска рост густота волос быстро отращивать вол...,233626208,233626208,2015-06-09 00:52:42,Светлана,Абрамова,Александров,Владимирская область,57,238,...,213.0,0.0,10.0,0.0,27.0,0.0,,94.0,32.0,4.0
